# Cognyx client debug notebook

This notebook is used to debug the Cognyx client used in cognyx automations.
you will need a .env file with a JWT token to run this notebook. You can get this after logging into the cognyx app.

In the cell below, define the bom_id and view_id you want to use (view_id will default to default view if not provided)

In [1]:
import os

jwt = os.getenv("COGNYX_TOKEN")

bom_id = "eca4a66c-43ab-4e44-b641-82ad9571672f"
view_id = None
base_url = "http://localhost:4000"

if not bom_id:
    raise ValueError("bom_id is required")

It will load the bom data and return it.

In [2]:
from cognyx_bom_sdk.cognyx import CognyxClient

cognyx = CognyxClient(base_url=base_url, jwt_token=jwt)

bom_data = await cognyx.load_bom_data(bom_id, view_id)

{'id': 'eca4a66c-43ab-4e44-b641-82ad9571672f', 'name': 'Automations Test', 'description': 'This bom demos automations', 'reference': 'AUTOMATIONS_TEST_1', 'image': '', 'status': 'draft', 'failure_reason': None, 'created_at': '2025-04-29T05:41:13.000000Z', 'updated_at': '2025-05-15T11:08:42.000000Z', 'auth': {'can': {'view': True, 'update': True, 'delete': True, 'share': True, 'comment': True, 'create': True, 'create_revision': True}}}


When the bom is loaded it will then initialize the BomClient

In [3]:
from cognyx_bom_sdk.client import BomClient


def update_callback(update: dict) -> None:
    """Callback function to handle updates to the BOM."""
    print("Update:", update)


client = BomClient(bom_data=bom_data, update_callback=update_callback)

In [4]:
instance = client.get_bom_instance_by_id("fe1ffe16-7a71-4060-b9f8-82f2780168fd")

print(instance.name)

client.set_instance_attribute(instance.id, "severity", 1)
client.set_instance_attribute(instance.id, "Wall thickness (mm)", 200)

await cognyx.update_instance_attribute(instance.id, "29a09f20-2e12-487b-860f-ede201add37f", 1)

await cognyx.update_instance_attribute(instance.id, "b02d1827-2714-412a-a470-330ddb38d3a3", 300)

Upper Assembly
Update: {'type': <BomUpdates.ATTRIBUTE_UPDATE: 'attribute_update'>, 'payload': {'instance_id': 'fe1ffe16-7a71-4060-b9f8-82f2780168fd', 'attribute_id': '29a09f20-2e12-487b-860f-ede201add37f', 'attribute_value': 1}}
Update: {'type': <BomUpdates.ATTRIBUTE_UPDATE: 'attribute_update'>, 'payload': {'instance_id': 'fe1ffe16-7a71-4060-b9f8-82f2780168fd', 'attribute_id': 'b02d1827-2714-412a-a470-330ddb38d3a3', 'attribute_value': 200}}
{'data': [{'id': 'fe1ffe16-7a71-4060-b9f8-82f2780168fd', 'entity_type_id': 'd032cb72-70fe-47a4-a685-b616304dff4e', 'sync_key': '6b7b3377-c2bc-4a33-bfbb-591c6c1c3c6d', 'reference': 'upper_assembly', 'name': 'Upper Assembly', 'description': 'Option F1 - Vibration welding', 'bom_id': 'eca4a66c-43ab-4e44-b641-82ad9571672f', 'parent_id': None, 'view_id': '2a39468f-c853-46f9-9d9b-ddd763b60263', 'quantity': 1, 'uom': 'pc', 'created_at': '2025-04-30T11:59:09.000000Z', 'updated_at': '2025-05-15T09:40:04.000000Z'}]}
{'data': [{'id': 'fe1ffe16-7a71-4060-b9f8-8

You can now use the client to interact with the BOM.

In [5]:
from functools import reduce

from cognyx_bom_sdk.models import BomInstance

mass_attribute = "added mass g"


def get_instance_mass(instance: BomInstance) -> int:
    """Get the mass of a BOM instance.

    Args:
        instance: BOM instance

    Returns:
        The mass of the instance
    """
    instance_mass = client.get_instance_attribute(instance.id, mass_attribute)

    if instance_mass is None:
        return 0

    print(f"instance {instance.name} weighs {instance_mass}g")
    return instance.quantity * instance_mass


reduce(lambda total, instance: total + get_instance_mass(instance), client.list_bom_instances(), 0)

0

In [6]:
instance = client.get_bom_instance("branch a")
attribute = client.get_instance_attribute(instance.id, "country")

result = await cognyx.update_instance(instance.id, {"quantity": 3})


attribute_change = await cognyx.update_instance_attribute(
    instance.id, "c782de11-0d87-4f07-91f3-9461edbbf3d2", "India"
)

attribute_value = None
for attr in attribute_change.custom_attributes:
    if attr.name.lower() == "country":
        attribute_value = attr.value
        break

attribute_value

AttributeError: 'NoneType' object has no attribute 'id'